In [ ]:
import os
os.chdir('..')

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import gc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb

import shap

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from scripts.utils import log_msg, precision_reall_f1_report

In [ ]:
args={}
args['data'] = 'data/sample_data_features.csv'
args['feature_space'] = 'data/feature_names.csv'
args['test_size'] = 0.2
args['seed'] = 123456

args['cv_results'] = 'results/cv_lightgbm.csv'

In [ ]:
args['NFOLDS'] = 3
args['num_iterations'] = [20, 40, 60, 80, 100]
args['early_stopping_rounds'] = 400
args['verbose'] = 0
args['n_jobs'] = 4
args['verbose_eval'] = 100

In [ ]:
data = pd.read_csv(args['data'])
data.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['sequence', 'label'], axis=1), 
                                                    data[['label']], 
                                                    test_size=args['test_size'], 
                                                    random_state=args['seed'])

### CV

In [ ]:
folds = KFold(n_splits=args['NFOLDS'], shuffle=True, random_state=args['seed'])

columns = X_train.columns
score = 0

feature_importances = pd.DataFrame()
feature_importances['feature'] = columns

results = []
log_msg(">>> Start CV")
for i in range(len(args['num_iterations'])):

    gc.collect()
    score = 0
    splits = folds.split(X_train, y_train)
    
    params = {'objective':'binary',
              'num_iterations': args['num_iterations'][i],
              'n_jobs': args['n_jobs'],
              'random_state': args['seed'],
              "metric": 'auc',
              'verbosity': args['verbose']
         }

    for fold_n, (train_index, valid_index) in enumerate(splits):

        X_train_cv, X_valid = X_train[columns].iloc[train_index], X_train[columns].iloc[valid_index]
        y_train_cv, y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]

        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dvalid = lgb.Dataset(X_valid, label=y_valid)

        clf = lgb.train(params, dtrain, valid_sets = [dvalid], verbose_eval=args['verbose_eval'], 
                        early_stopping_rounds=args['early_stopping_rounds'])

        y_pred_valid = clf.predict(X_valid)

        score += roc_auc_score(y_valid, y_pred_valid) / args['NFOLDS']
        
        del X_train_cv, X_valid, y_train_cv, y_valid
        del clf, y_pred_valid

    log_msg(f">>> num_iterations = {args['num_iterations'][i]} Mean AUC = {score}")
    
    results.append({'num_iterations': args['num_iterations'][i],
               'score': score})
    
log_msg(">>> Finished!")

results_df = pd.DataFrame(results)
results_df.head()

In [ ]:
X_train_final, X_valid, y_train_final, y_valid = train_test_split(X_train, y_train, 
                                                                  test_size=0.1, random_state=args['seed'])

params = {'objective':'binary',
          'num_iterations': 100,
          'n_jobs': 4,
          'random_state': args['seed'],
          "metric": 'auc',
          'verbosity': args['verbose']
         }

dtrain = lgb.Dataset(X_train_final, label=y_train_final)
dvalid = lgb.Dataset(X_valid, label=y_valid)

clf = lgb.train(params, dtrain, valid_sets = [dvalid], verbose_eval=args['verbose_eval'], 
                early_stopping_rounds=args['early_stopping_rounds'])

feature_importances = pd.DataFrame()
feature_importances['feature'] = X_train.columns

feature_importances[f'final_train'] = clf.feature_importance()

In [ ]:
y_pred = clf.predict(X_valid)

precision, recall, thresholds = precision_recall_curve(y_valid, y_pred)

reports = precision_reall_f1_report(precision, recall, thresholds, 
                                    font_scale=2,
                                    linewidth=3,
                                    plot=False)

threshold = reports[reports['f1']==reports['f1'].max()]['threshold'].values[0]

print('Threshold to get the best F1 on validation set: ', threshold)

In [ ]:
y_pred = clf.predict(X_test)

print(f">>> AUC on Test set: {roc_auc_score(y_test, y_pred)}\n")

y_pred_label = [1 if i >= threshold else 0 for i in y_pred]

print(f">>> F1 on Test set (threshold {threshold}) : {f1_score(y_test, y_pred_label)}\n")


### Precision, Recall and F1 vs threshold on test set

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)

In [ ]:
reports = precision_reall_f1_report(precision, recall, thresholds, 
                                    font_scale=2,
                                    linewidth=3)

In [ ]:
reports.head()

In [ ]:
print('Best F1: ', reports['f1'].max())
print('Threshold:', reports[reports['f1']==reports['f1'].max()]['threshold'].values[0])

In [ ]:
feature_space = pd.read_csv(args['feature_space'], dtype={'feature': str})
feature_importances['pattern'] = feature_space['feature'].values
feature_importances.head()

### Feature importance

In [ ]:
# Shap values
explainer = shap.TreeExplainer(clf)
shap_values = explainer(X_train)


In [ ]:
shap_values.feature_names = feature_importances['pattern'].tolist()

In [ ]:
shap.summary_plot(shap_values[:, :, 1], max_display=20)